<a href="https://colab.research.google.com/github/ckim415/OOOA/blob/main/recomendation_system_flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

# 구글 드라이브 파일 확인
!ls '/gdrive/My Drive/프로젝트2'

# 반복되는 드라이브 경로 변수화
drive_path = '/gdrive/My Drive/프로젝트2/'

Mounted at /gdrive
 6144_link.csv		       타이틀전처리.ipynb
 archive.zip		       kobert.csv
 base_20000.txt		       kogpt2.csv
'추천시스템 by 타이틀.ipynb'   kogpt2.csv.txt
 cate.csv		       kogpt2_try.ipynb
 cate_keyword.csv	       kogpt2_wadiz.ipynb
 concat_df.csv		       recomendation_system_flask.ipynb
 crawling_final.ipynb	       text_process.h5
 df_added.csv		       title.txt
 df_nouns_add.csv	       tokenizer.pickle
 df_pos.csv		       try_kobert.ipynb
 df_reco_system.csv	       Untitled0.ipynb
 edu_link_df		       Untitled1.ipynb
 game_df.csv		       Untitled2.ipynb
 home.csv		       wadizlauncher.tgz
'프로젝트 데이터 분석.ipynb'


In [2]:
!pip install konlpy wordcloud

     |████████████████████████████████| 19.4MB 1.5MB/s 
     |████████████████████████████████| 92kB 8.2MB/s 
     |████████████████████████████████| 460kB 25.6MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [3]:
!pip install krwordrank

In [8]:
!pip install flask-ngrok

In [7]:
!pip install emoji

     |████████████████████████████████| 133kB 8.4MB/s 


# 추천 시스템 

In [5]:
import pandas as pd
import numpy as np
import re
from string import punctuation
from konlpy.tag import Kkma, Hannanum, Komoran, Mecab, Okt
# from krwordrank.word import KRWordRank
# from krwordrank.hangle import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
df = pd.read_csv(drive_path + 'concat_df.csv')

In [46]:
df['title'][50:100]

50         3400%앵콜⭐2 in 1 천연향수&여성/남성청결제, 위생·향기
51               쏘헤어가 만든 또 하나의 명작! 탈모방지 스칼프 샴푸
52    [리뉴얼 앵콜/평점종결] 바쁜아침 하나로-고순도 이데베논 #깨죽크림2.0
53         앵콜ㅣ주름 걱정 그만! 비밀은 동백나무꽃추출물? 7초간의 머무름
54     [2731%앵콜]완벽한 결과 톤의 비밀, 보르피린앰플395,000ppm
55      "피부장벽개선" 임상입증!! 피부고민 더 이상 손대지 말고 찌르세요!
56                 [1억신화/평점4.9앵콜] 미친발색 글리터 섀도우
57        [혁신 트렌디 핸드겔] 에탄올이 들어가 있는데 보습감과 쿨링감도?
58    1억 달성 꿀광템| 후기가 인증하는 439%흡수율! 닥터EP의 마지막앵콜
59                  아직도 흔들어 써?! 1타 3피 젤리로 뿌려!!
60    [후원] 어른도 탐내는 유아용크림, <현's크림>. 아빠가 만드니 다르다
61       [앵콜] 냉각기능 더해 모공까지 닫아주는 냉온갈바닉 발키리 아이스킨
62                 모든 화학물질로부터 자유로운 청결티슈 'ZERO'
63                     24시간 올데이 트러블 케어, 닥터스팟패치
64            정말 믿을만한 줄기세포 배양액 화장품을 기다려왔던 당신에게
65                 오가닉박스와 함께하는 안전한 화장품을 위한 캠페인
66         탈모 관리의 시작, 특허성분과 해초 추출물 첨가된 <캘프샴푸S>
67     [1억신화/2차 앵콜 아이크림] 단하루,1캡슐 사용만으로 달라지는 피부
68        백화점 화장품, 내용물만 살 수 없을까? 30초 각질케어 필링토너
69         다시어터를 구원하다! : 세상 맛있는 신개념 다이어트&쾌변 카페
70        마스크 착용 중? 피부 관리 중! 마스크 안에 붙이는 이너마스크팩
71           

In [7]:
#함수화
def clean(text):
  pattern2 = re.compile(r"\d+.?,?\d+[ㄱ-ㅎ가-힣]+") # 숫자 ., remove ^
  pattern1 = re.compile(r'\d+%') # 숫자% ^
  pattern3 = re.compile(r'\d+[a-zA-Zㄱ-ㅎ가-힣]+') #숫자 + 문자
  pattern4 = re.compile(r'[0-9]') #숫자 제거
  pattern5 = re.compile(r'[{}]'.format(re.escape(punctuation))) #punc 제거
  pattern6 = re.compile(r'[a-zA-Z]') #영어 제거
  pattern7 = re.compile(r'[^ㄱ-ㅎ가-힣_]')
  pattern8 = re.compile(r'\s{2,5}') #띄어쓰기 2개 이상 하나로

  text = pattern2.sub('', str(text))
  text = pattern1.sub('', text)
  text = pattern4.sub('', text)
  text = pattern3.sub('', text)
  text = pattern5.sub(' ', text)
  text = pattern6.sub('', text)
  text = pattern7.sub(' ', text)
  text = pattern8.sub(' ', text)
  return text.strip()

In [8]:
df['title_clean'] = df['title'].apply(clean)
df['title_clean'][50:100]

50                   앵콜 천연향수 여성 남성청결제 위생 향기
51             쏘헤어가 만든 또 하나의 명작 탈모방지 스칼프 샴푸
52       리뉴얼 앵콜 평점종결 바쁜아침 하나로 고순도 이데베논 깨죽크림
53         앵콜 주름 걱정 그만 비밀은 동백나무꽃추출물 초간의 머무름
54                   앵콜 완벽한 결과 톤의 비밀 보르피린앰플
55        피부장벽개선 임상입증 피부고민 더 이상 손대지 말고 찌르세요
56                      억신화 평점 미친발색 글리터 섀도우
57        혁신 트렌디 핸드겔 에탄올이 들어가 있는데 보습감과 쿨링감도
58          억 달성 꿀광템 후기가 인증하는 흡수율 닥터의 마지막앵콜
59                     아직도 흔들어 써 타 피 젤리로 뿌려
60        후원 어른도 탐내는 유아용크림 현 크림 아빠가 만드니 다르다
61      앵콜 냉각기능 더해 모공까지 닫아주는 냉온갈바닉 발키리 아이스킨
62                     모든 화학물질로부터 자유로운 청결티슈
63                        올데이 트러블 케어 닥터스팟패치
64         정말 믿을만한 줄기세포 배양액 화장품을 기다려왔던 당신에게
65              오가닉박스와 함께하는 안전한 화장품을 위한 캠페인
66          탈모 관리의 시작 특허성분과 해초 추출물 첨가된 캘프샴푸
67       억신화 차 앵콜 아이크림 단하루 캡슐 사용만으로 달라지는 피부
68           백화점 화장품 내용물만 살 수 없을까 각질케어 필링토너
69         다시어터를 구원하다 세상 맛있는 신개념 다이어트 쾌변 카페
70       마스크 착용 중 피부 관리 중 마스크 안에 붙이는 이너마스크팩
71                    고단한 일상에 잠깐의 힐링 테라피 샴푸
72     꽃잎의 아름다움이 피부로 스르륵 내일은 더 아

In [9]:
def noun(sentence):
  kkma = Kkma()
  nouns = kkma.nouns(sentence)

  return nouns

df['nouns'] = df['title_clean'].apply(noun)

In [10]:
df['nouns'][50:100]

50    [앵, 앵콜, 콜, 천연, 천연향수, 향수, 여성, 남성, 남성청결제, 청결제, 위...
51                 [쏘헤어, 하나, 명작, 탈모, 탈모방지, 방지, 스칼프, 샴푸]
52    [리, 리뉴얼, 뉴, 얼, 앵, 앵콜, 콜, 평점, 평점종결, 종결, 아침, 하나,...
53    [앵, 앵콜, 콜, 주름, 걱정, 비밀, 동백나무, 동백나무꽃추출물, 꽃, 추출물,...
54      [앵, 앵콜, 콜, 완벽, 결과, 톤, 비밀, 보, 보르피린앰플, 르, 피, 린앰플]
55     [피부, 피부장벽개선, 장벽, 개선, 임상, 임상입증, 입증, 피부고민, 고민, 이상]
56                              [억, 억신화, 신화, 평점, 발색, 섀]
57    [혁신, 트렌, 트렌디, 디, 핸드, 핸드겔, 겔, 에탄올, 있는데, 보습, 보습감...
58    [억, 달, 달성, 성, 꿀, 꿀광템, 광, 템, 후, 후기가, 기가, 인증, 흡수...
59                                              [피, 젤리]
60                              [후원, 어른, 유아, 크림, 현, 아빠]
61    [앵, 앵콜, 콜, 냉각, 냉각기능, 기능, 모공, 냉온, 바, 바닉, 닉, 발키리...
62                 [모든, 화학, 화학물질, 물질, 자유, 청결, 청결티슈, 티슈]
63              [올, 올데이, 데이, 트러블, 케어, 닥터, 닥터스팟패, 스팟, 패]
64                         [줄기, 줄기세포, 세포, 배양액, 화장품, 당신]
65                                [오가닉박스, 안전, 화장품, 캠페인]
66    [탈모, 관리, 시작, 특허, 특허성분, 성분, 해초, 추출물, 첨가, 프, 프샴푸...
67    [억, 억신화, 신화, 차, 앵, 앵콜, 콜, 아이, 아이크림, 크림, 단하

In [11]:
df.to_csv(drive_path + 'reco_df.csv')

## 린넨셔츠 비교1
- case1

In [96]:
def find_title(df, noun, top_n=10):
  noun = '린넨셔츠'
  df['in_nouns_literal'] = df['nouns'].apply(lambda x: True if noun in x else False)
  df['in_nouns_literal'].value_counts()
  title_index = df[df['in_nouns_literal'] == True].index.values
  new_df = df.iloc[title_index][['categories','title','funding_amounts','likes']]

  new_df = new_df[new_df['categories']==new_df['categories'].value_counts().index[0] ]

  C = new_df['funding_amounts'].mean()
  m = new_df['likes'].quantile(0.6)
  C, m

  def weighted_funding_average(record):
    v = record['likes']
    R = record['funding_amounts']
    return ( (v/(v+m)) * R ) + ((m/(v+m)) * C)

  new_df['weighted_funding'] = new_df.apply(weighted_funding_average, axis=1)
  new_df['weighted_funding'] = new_df['weighted_funding'].apply(lambda x: round(x,2))
  return new_df.sort_values('weighted_funding', ascending=False)[:top_n]

,categories,title,funding_amounts,likes,weighted_funding
13941,패션·잡화,"검증된 21번째 펀딩ㅣ여름의 품격, 입을수록 부드러운 벨지움 린넨셔츠",15287000,120,11931063.61
14426,패션·잡화,"정직한 가격, 최상급 소재의 린넨셔츠",10883000,60,8507562.25
14044,패션·잡화,덥고 땀나는 여름에 탁월한 선택 냄새제거 린넨셔츠,7071000,108,6889763.41
17040,패션·잡화,[셔츠의정석 4탄] 린넨셔츠는 왜 여름에만? 4계절 입는 오픈카라 셔츠!,1253000,11,5952085.82
17594,패션·잡화,"골라입는 즐거움, 6개의 사이즈 8개 색상으로 무장한 #린넨셔츠",1887300,34,5165606.06
16457,패션·잡화,[앵콜] 한여름을 책임질 환경 친화적 린넨셔츠를 2만원 초반대에 입자!!,3412000,76,5022108.33


## 결과
- 1번이 반복되지 않고 깔끔하게 나온다.

## 린넨셔츠 비교2

In [66]:
df['nouns_literal'] = df['nouns'].apply( lambda x : (' ').join(x))
#형변환 
df['nouns_literal'] = df['nouns_literal'].astype('str')
df['nouns_literal']

0                       와디 와디즈 즈 최초 딸 엄마 주름 주름개선 개선 만능 크림
1        앵 앵콜 콜 하루 수분 수분증가 증가 임상 임상검증 검증 완료 바 바닉마스크 닉 마스크
2                             유명 피부과 공급 시크 시크릿 릿 포뮬러 프로그램
3                    두피 관리 핵심 유 유분량 분량 균형 솔 모닝 나이트 밸런스 샴푸
4                                  컨디셔너 필요 샴푸 하나 두피 모발 건강
                               ...                       
21475                         히트 히트패드 패드 출근길 핸들 전기 핫 핫팩 팩
21476       무선 에어 에어펌프 펌프 자동차 오토바이 오토바이공기압 공기압 자전거 튜브 축구공
21477                       나 반응 반응형 형 웹 웹페이지 페이지 제작 실크로드
21478                        웨어 웨어러 러 공기 공기청정기 청정기 전동 마스크
21479                                                  시계
Name: nouns_literal, Length: 21480, dtype: object

In [ ]:
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))           
nouns_mat = count_vect.fit_transform( df['nouns_literal'] )
nouns_mat.shape

# 코사인 유사도 계산계산
nouns_sim = cosine_similarity(nouns_mat, nouns_mat)

# 유사도가 높은 순으로 정리된 genre_sim 객체의 비교 행 위치 인덱스 값
nouns_sim_sorted_ind = nouns_sim.argsort()[:, ::-1]
nouns_sim_sorted_ind[:1]


In [80]:

def find_sim_title(df, sorted_ind, noun, top_n=10):
  df['in_nouns_literal'] = df['nouns'].apply(lambda x: True if noun in x else False)
  title_index = df[df['in_nouns_literal'] == True].index.values
  sim_indexs = sorted_ind[title_index, :(top_n)]
  sim_indexs = sim_indexs.reshape(-1)
  new_df = df.iloc[sim_indexs]
  new_df = new_df[new_df['categories']==new_df['categories'].value_counts().index[0] ]

  C = new_df['funding_amounts'].mean()
  m = new_df['likes'].quantile(0.6)
  C, m

  def weighted_funding_average(record):
    v = record['likes']
    R = record['funding_amounts']
    return ( (v/(v+m)) * R ) + ((m/(v+m)) * C)

  new_df['weighted_funding'] = new_df.apply(weighted_funding_average, axis=1)
  new_df['weighted_funding'] = new_df['weighted_funding'].apply(lambda x: round(x,2))

  return new_df.sort_values('weighted_funding', ascending=False)[:top_n]

In [97]:
sim_title = find_sim_title(df, nouns_sim_sorted_ind, '린넨셔츠')
sim_title[['title','categories','funding_amounts','n_supporters']]

,title,categories,funding_amounts,n_supporters
13103,"[벌써 22번째 펀딩] 여름의 시원한 품격, 벨지움 린넨 셔츠",패션·잡화,40461500,316
13103,"[벌써 22번째 펀딩] 여름의 시원한 품격, 벨지움 린넨 셔츠",패션·잡화,40461500,316
12882,"[검증된 20번째 펀딩] 여름의 품격, 프리 다잉 벨지움 린넨 셔츠",패션·잡화,33527000,242
13645,"[앵콜] 20번째 펀딩 l 여름의 시원한 품격, 벨지움 린넨 셔츠",패션·잡화,20224100,144
12515,"[앵콜] 여름대비 6만원으로 끝! 자외선차단, 닿는 순간 쿨한 쿨스킨점퍼",패션·잡화,17107500,214
13941,"검증된 21번째 펀딩ㅣ여름의 품격, 입을수록 부드러운 벨지움 린넨셔츠",패션·잡화,15287000,120
13941,"검증된 21번째 펀딩ㅣ여름의 품격, 입을수록 부드러운 벨지움 린넨셔츠",패션·잡화,15287000,120
13941,"검증된 21번째 펀딩ㅣ여름의 품격, 입을수록 부드러운 벨지움 린넨셔츠",패션·잡화,15287000,120
13941,"검증된 21번째 펀딩ㅣ여름의 품격, 입을수록 부드러운 벨지움 린넨셔츠",패션·잡화,15287000,120
13941,"검증된 21번째 펀딩ㅣ여름의 품격, 입을수록 부드러운 벨지움 린넨셔츠",패션·잡화,15287000,120


# flask

In [17]:

from flask import Flask,request, url_for, render_template
import numpy as np
import random
import datetime
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize
from konlpy.tag import Kkma, Hannanum, Komoran, Mecab, Okt
import re
from string import punctuation
from tensorflow.keras.models import load_model
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from flask_ngrok import run_with_ngrok
import tensorflow as tf
import pickle

app = Flask(__name__)
run_with_ngrok(app)  # 코랩일 경우 필요함.


model = load_model(drive_path + 'text_process.h5')
# loading
with open(drive_path + 'tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)


@app.route("/", methods=['GET', 'POST'])
def index():
    if request.method == 'GET':
      return render_template('index.html')
    if request.method == 'POST':

      # 파라미터를 전달 받습니다. 입력 데이터로 받은 값이 X_data에 담김. 
      X_data = str(request.form['X_data'])
      
      # 단어 전처리 과정을 거쳐야겠지? // 안해도됨
      # X_data = keyword(X_data)
      # X_data = noun(X_data)

      token = Tokenizer()

      # 문자열 정수 인덱스 리스트 출력
      
      # sequence의 길이를 20으로 통일. 제일 긴 단어 갯수가 20이라서.
      sequences = token.texts_to_sequences(X_data)
      padding_length = 10
      x_padded = pad_sequences(sequences, padding_length)

      prediction_value = format(model.predict(x_padded)[0][0]*100, ".2f")
      print('------>', prediction_value)

      return render_template('index.html', prediction_value= prediction_value)
if __name__ == '__main__':    
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://714dab335108.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


[2021-05-21 14:10:52,581] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-17-edca2af70c6b>", line 31, in index
    return render_template('index.html')
  File "/usr/